<a href="https://colab.research.google.com/github/taehyeong51/LAMO/blob/main/LAMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **당신의 더욱 완벽한 추억을 위해**

Mask2Former(MMDetection) : https://github.com/open-mmlab/mmdetection.git

Lama : https://github.com/saic-mdal/lama

<img src = "https://drive.google.com/uc?id=1AsWI5RN66aQEprOnNQj_ryV27-YS8NNJ" height = 300 width = 1000>

이미지 내 불필요한 인물 객체를 제거하여 당신이 원하는 더욱 완벽한 사진을 생성할 수 있습니다.

[MMDetection](https://github.com/open-mmlab/mmdetection.git) 기반의 [Mask2Former](https://github.com/open-mmlab/mmdetection/tree/master/configs/mask2former)를 기반으로 인물을 탐지하여 mask를 생성하고 [LaMa](https://github.com/saic-mdal/lama) 를 기반으로 mask 영역의 이미지를 새롭게 생성하였습니다.




In [ ]:
#@title <b><font color="black">MMDetection 진행을 위한 환경 구축</b>
#@markdown 3-5분 정도 소요됩니다

import time
start = time.time()
!pip3 install rich
!pip3 install modelindex

!mim install mmcv-full

%cd /content
!pip3 install torch
!pip install torchvision
!pip3 install rich
!pip3 install modelindex
!pip3 install openmim
!mim install mmcv-full

!git clone https://github.com/open-mmlab/mmdetection.git

%cd /content/mmdetection

!pip install -r requirements/build.txt
!pip install -v -e .

# download pth(checkpoint) file of Mask2Former
!gdown https://download.openmmlab.com/mmdetection/v2.0/mask2former/mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco/mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco_20220504_001756-743b7d99.pth
!mkdir /content/mmdetection/checkpoints
!mv /content/mmdetection/mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco_20220504_001756-743b7d99.pth /content/mmdetection/checkpoints

!mkdir /content/input_lama
!mkdir /content/result
print("MMDetection 환경 구축 소요시간 : {}분".format(round((time.time() - start)/60,2)))

In [ ]:
#@title <b><font color="black">LaMa 환경 구축</b>
#@markdown 6-7분 정도 소요됩니다

import time
start = time.time()
%cd /content
# Run this sell to set everything up
print('\n> Cloning the repo')
!git clone https://github.com/saic-mdal/lama.git

print('\n> Install dependencies')
!pip install torch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 torchtext==0.9
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html --quiet


print('\n> Changing the dir to:')
%cd /content/lama
!export TORCH_HOME=$(pwd) && export PYTHONPATH=$(pwd)
# % export TORCH_HOME=$(pwd) && export PYTHONPATH=$(pwd)



print('\n> Download the model')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
!unzip big-lama.zip

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet

%cd /content/lama
!export TORCH_HOME=$(pwd) && export PYTHONPATH=$(pwd)

print("LaMa 환경 구축 소요시간 : {}분".format(round((time.time() - start)/60,2)))

In [ ]:
#@title <b><font color="black">segmentation 모델 설정 → Mask2Former</b>
% cd /content/mmdetection
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from google.colab.patches import cv2_imshow
import mmcv
import numpy as np
import cv2
import pandas as pd

config_file = 'configs/mask2former/mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco.py'
checkpoint_file = 'checkpoints/mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco_20220504_001756-743b7d99.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# <b>이미지 준비</b>
이미지를 업로드 해주세요.

In [ ]:
#@title <b><font color="black">이미지 업로드</b>

from google.colab import files
import shutil 

def upload_file():
    uploaded = files.upload()
    try:
        fn = list(uploaded.keys())[0]
    except:
        print ("Please upload a valid image file!")
        raise StopExecution
    print('Uploaded file "{name}" of {length} bytes'.format(name=fn,length=len(uploaded[fn])))
    return fn


print ("이미지를 업로드 해 주세요")
fn = upload_file()
shutil.move(fn, "/content/mmdetection/demo/test.jpg")

# <b>인물 선택</b>
이미지 내에서 제거할 인물을 선택해주세요.

In [ ]:
#@title <b><font color="black">인물 선택</b>

from ipywidgets import Button, HBox, VBox, widgets, Checkbox

img = '/content/mmdetection/demo/test.jpg'
name = img.split("/")[4].split(".")[0]
original = cv2.imread(img)
shape_ori = original.shape

result = inference_detector(model, img)
show_result_pyplot(model, img, result)

original_img = cv2.imread(img)
analysis = np.array(result)
processed_img = original_img.copy()

### 사람찾기

### people 의 신뢰도만 갖는 np array 생성
n = len(analysis[0][0])
list_people = [0 for i in range(n)]

for i in range(len(analysis[0][0])):
  if analysis[0][0][i][4] > 0.3:
    list_people[i] = analysis[0][0][i][4]


shape = original_img.shape
mask_img = np.zeros((shape), dtype = np.uint8)
### range(n), n에 해당하는 만큼의 대상 delete

def gonogo(gonogo_img, people, information):
  cnt = 0
  for i in range(len(people)):


    guy = max(people)
    idx = people.index(guy)
    if guy < 0.2:
      pass
    else:
      xmin,ymin,xmax,ymax = map(round,information[0][0][idx][:4])
      cnt += 1

      target = information[1][0][idx]*1
      (w, h), _ = cv2.getTextSize(
        "person {}".format(int(cnt)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
      
      blue = (255, 0, 0)
      font =  cv2.FONT_HERSHEY_PLAIN
      if i // 2 == 0:
        y = -10
      else:
        y = 30
      gonogo_img = cv2.putText(gonogo_img, "person {}".format(int(cnt)), (int((xmax+xmin)/2-30), ymin+y), font, 1, (255,255,255), 1, cv2.LINE_AA)
      
      people[idx] = 0
  return gonogo_img, cnt


gonogo_img, p_cnt = gonogo(processed_img, list_people, analysis)
cv2_imshow(gonogo_img)

output = widgets.Output()
print ("")
print ("")
print ("제거할 인물의 인덱스를 선택해주세요")
numbers = [widgets.Checkbox(value=False, description=str(i+1)) for i in range(p_cnt)]
output = widgets.VBox(children=numbers)
display(output)

# Mask 생성

In [ ]:
#@title <b><font color="black"> instance segmentation</b>


selected_data = []
for i in range(0, len(numbers)):
    if numbers[i].value == True:
        selected_data.append(int([numbers[i].description][0])-1)

name = img.split("/")[4].split(".")[0]
original = cv2.imread(img)
shape_ori = original.shape

### 변수선언
original_img = cv2.imread(img)
analysis = np.array(result)
processed_img = original_img.copy()

### 사람찾기

### people 의 신뢰도만 갖는 np array 생성
n = len(analysis[0][0])
list_people = [0 for i in range(n)]

for i in range(len(analysis[0][0])):
  if analysis[0][0][i][4] > 0.3:
    list_people[i] = analysis[0][0][i][4]


shape = original_img.shape
mask_img = np.zeros((shape), dtype = np.uint8)
### range(n), n에 해당하는 만큼의 대상 delete
cnt = 0
for i in range(len(list_people)):
                        
  
  guy = max(list_people)
  idx = list_people.index(guy)
  if i not in selected_data:
    pass
  else:
    cnt += 1

    target = analysis[1][0][idx]*1
    mask = np.array(target.copy(), dtype = np.uint8) # 0~255로 변환
    mask_reverse = np.array(analysis[1][0][idx]*255, dtype=np.uint8)

    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

    dst = cv2.dilate(mask_img, k)
    
    ch_b = original_img[:,:,0]
    ch_g = original_img[:,:,1]
    ch_r = original_img[:,:,2]

    filtered_img = cv2.merge((ch_b*mask, ch_g*mask, ch_r*mask)) # target only
    filtered_img2 = cv2.merge((mask*255, mask*255, mask*255))

    filtered_img = cv2.dilate(filtered_img,k)
    filtered_img2 = cv2.dilate(filtered_img2,k)

    processed_img = cv2.addWeighted(processed_img ,1, filtered_img2, 0.5,0)
    
    mask1d_img = mask_img[:,:,0] + mask_reverse
    

    mask_reverse = cv2.merge((mask_reverse,mask_reverse,mask_reverse))
    mask_img = mask_img + mask_reverse

  list_people[idx] = 0
  
mask_img = cv2.dilate(mask_img,k)
mask1d1_img = cv2.dilate(mask1d_img,k)
mask1d2_img = cv2.dilate(mask1d1_img, k, iterations = 10)

cv2.imwrite("/content/result/masked_img.png", processed_img)
cv2.imwrite("/content/result/mask_img.png", mask_img)

cv2.imwrite("/content/result/{}.png".format(name),original_img )

# for LaMa

cv2.imwrite("/content/input_lama/{}_mask01.png".format(name), mask1d1_img)
cv2.imwrite("/content/input_lama/{}_mask02.png".format(name), mask1d2_img)
cv2.imwrite("/content/input_lama/{}.png".format(name),original_img )

print("complete :)")


In [ ]:
#@title <b><font color="black">object detection</b>

### 변수선언
original_img = cv2.imread(img)
analysis = np.array(result)
processed_img = original_img.copy()
# showme(processed_img)

### 사람찾기

### people 의 신뢰도만 갖는 np array 생성
n = len(analysis[0][0])
list_people = [0 for i in range(n)]

for i in range(len(analysis[0][0])):
  if analysis[0][0][i][4] > 0.5:
    list_people[i] = analysis[0][0][i][4]

shape = original_img.shape
mask_img = np.zeros((shape), dtype = np.uint8)
### range(n), n에 해당하는 만큼의 대상 delete
cnt = 0
for i in range(len(list_people)):
  guy = max(list_people)
  idx = list_people.index(guy)
  if i not in selected_data:
    pass
  else:
    cnt += 1
    ## bounding box procedure
    xmin,ymin,xmax,ymax = map(round,analysis[0][0][idx][:4])
    ymax = int(ymax * float(1.01))
    white = (255,255,255)

    mask_img = cv2.rectangle(mask_img, (xmin, ymin), (xmax, ymax),white,-1 )
    invert_img = np.invert(mask_img)
    if i == 0:
      processed_img = cv2.bitwise_and(original_img, invert_img)
    else:
      processed_img = cv2.bitwise_and(processed_img, invert_img)
    


  list_people[idx] = 0
  


# for mmediting

cv2.imwrite("/content/result/bbox_masked_img.png", processed_img)
cv2.imwrite("/content/result/bbox_mask_img.png", mask_img)

# for LaMa
cv2.imwrite("/content/input_lama/{}.png".format(name), original_img)
cv2.imwrite("/content/input_lama/{}_mask03.png".format(name), mask_img)


print("complete :)")

# Inpainting by LaMa

In [ ]:
#@title <b><font color="black">Inpainting by LaMa</b>


% cd /content/lama

!export TORCH_HOME=$(pwd) && export PYTHONPATH=$(pwd)

# !python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/test_lama outdir= /content/result/result_bbox_inpainting_lama.png
!PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=/content/input_lama outdir=/content/result



# 결과


In [ ]:
#@title <b><font color="black">Inpainting 결과 확인</b>


original = cv2.imread("/content/result/test.png")
result = cv2.imread("/content/result/test_mask02.png")

cv2_imshow(original)
cv2_imshow(result)

# Re-inpainting

In [ ]:
#@title <b><font color="black">다시 mask 적용 후 inpainting</b>

!mkdir /content/input_lama2
!mkdir /content/result2

!cp -r /content/result/test_mask02.png /content/input_lama2/test_seg.png
!cp -r /content/result/test_mask03.png /content/input_lama2/test_bbox.png

!cp -r /content/input_lama/test_mask02.png /content/input_lama2/test_seg_mask01.png
!cp -r /content/input_lama/test_mask02.png /content/input_lama2/test_bbox_mask01.png

!cp -r /content/input_lama/test_mask03.png /content/input_lama2/test_seg_mask02.png
!cp -r /content/input_lama/test_mask03.png /content/input_lama2/test_bbox_mask02.png

%cd /content/lama
!export TORCH_HOME=$(pwd) && export PYTHONPATH=$(pwd)

# !python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/test_lama outdir= /content/result/result_bbox_inpainting_lama.png
!PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=/content/input_lama2 outdir=/content/result2


# Re-Inpainting 결과 확인

In [ ]:
#@title <b><font color="black">Re-inpainting 결과 확인</b>

import time

result = cv2.imread("/content/result/test_mask02.png")
result2 = cv2.imread("/content/result2/test_seg_mask02.png")

cv2_imshow(original)
print("original")
# time.sleep(3)

cv2_imshow(result)
print("Processed once")
# time.sleep(3)

cv2_imshow(result2)
print("Processed Twice")
